In [ ]:
import os
import glob
import cv2
import numpy as np
import torch
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from ultralytics import YOLO
import mediapipe as mp
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

# Paths - adjust to your local setup
# Paths - adjust to your local setup
INPUT_DIR = 'Time_12-34'              # folder containing JPEG frames
OUTPUT_DIR = 'output_segment' 
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Device for PyTorch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load YOLOv8n for person detection
yolo_model = YOLO('yolov8n.pt')

# Load Clothing Segmentation Model (e.g., substitute for FashionNet)
from torchvision.models.segmentation import lraspp_mobilenet_v3_large

def load_clothing_segmentation_model():
    model = lraspp_mobilenet_v3_large(pretrained=True)
    model.to(device)
    model.eval()
    return model

clothing_model = load_clothing_segmentation_model()

# Predict clothing mask using segmentation output (e.g., label 5 = shirt, 6 = pants)
def get_clothing_mask(image):
    with torch.no_grad():
        input_tensor = seg_transform(image).unsqueeze(0).to(device)
        output = clothing_model(input_tensor)['out']
        pred = torch.argmax(output.squeeze(), dim=0).cpu().numpy()
        clothing_mask = (pred == 5) | (pred == 6) | (pred == 7)  # shirt, pants, dress
        return clothing_mask.astype(bool)

# Placeholder for now - replace or integrate real model in place of DeepLabV3
def load_segmentation_model():
    model = deeplabv3_resnet50(pretrained=True, progress=True)
    model.to(device)
    model.eval()
    return model

segmentation_model = load_segmentation_model()

# Preprocessing transform for segmentation
seg_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((520, 520)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose.Pose(static_image_mode=True)
mp_drawing = mp.solutions.drawing_utils

# Segmentation mask from current model
def get_person_mask_resnet(crop):
    img = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    inp = seg_transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        out = segmentation_model(inp)['out'][0]
        mask = torch.sigmoid(out[15]).cpu().numpy() > 0.5
    return cv2.resize(mask.astype(np.uint8), (crop.shape[1], crop.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)

# Extract pose landmarks
def get_landmarks(crop):
    rgb = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
    return mp_pose.process(rgb).pose_landmarks

# Split body parts
def split_parts(mask, landmarks):
    H, W = mask.shape
    coords = {mp.solutions.pose.PoseLandmark(idx).name: (int(lm.x * W), int(lm.y * H)) for idx, lm in enumerate(landmarks.landmark)}
    hip_y = int((coords['LEFT_HIP'][1] + coords['RIGHT_HIP'][1]) / 2)
    rows = np.arange(H)[:, None]
    upper = mask & (rows < hip_y)
    lower = mask & (rows >= hip_y)
    hands = np.zeros_like(mask, dtype=bool)
    for side in ['LEFT', 'RIGHT']:
        key = f'{side}_WRIST'
        if key in coords:
            wx, wy = coords[key]
            r = int(0.1 * H)
            hands[max(0, wy-r):min(H, wy+r), max(0, wx-r):min(W, wx+r)] = True
    return upper, lower, hands, coords

# Plot and save HSV histograms
def plot_histogram(region_img, mask, title, save_path=None):
    hsv = cv2.cvtColor(region_img, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0], mask.astype(np.uint8), [180], [0, 180])
    plt.figure(figsize=(4, 2))
    plt.title(title)
    plt.xlabel("Hue")
    plt.ylabel("Frequency")
    plt.plot(hist, color='orange')
    plt.xlim([0, 180])
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.close()

# Feature extraction: refine region_mask using FashionNet-style clothing mask before histograms

def extract_clothing_features(region_img, region_mask):
    # Apply clothing-only mask
    clothing_mask = get_clothing_mask(region_img)
    clothing_mask = cv2.resize(clothing_mask.astype(np.uint8), (region_img.shape[1], region_img.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)
    refined_mask = region_mask & clothing_mask

    region = cv2.bitwise_and(region_img, region_img, mask=refined_mask.astype(np.uint8))
    hsv = cv2.cvtColor(region, cv2.COLOR_BGR2HSV)
    hist_h = cv2.calcHist([hsv], [0], refined_mask.astype(np.uint8), [16], [0, 180])
    hist_s = cv2.calcHist([hsv], [1], refined_mask.astype(np.uint8), [16], [0, 256])
    hist_v = cv2.calcHist([hsv], [2], refined_mask.astype(np.uint8), [16], [0, 256])
    color_hist = np.concatenate([hist_h, hist_s, hist_v]).flatten()
    color_hist /= (np.sum(color_hist) + 1e-6)
    gray = rgb2gray(region)
    lbp = local_binary_pattern(gray, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp[refined_mask], bins=10, range=(0, 10), density=True)
    return np.concatenate([color_hist, lbp_hist])

# GLCM texture features
def get_texture_features(gray_img):
    glcm = graycomatrix((gray_img * 255).astype(np.uint8), distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    return tuple(graycoprops(glcm, prop)[0, 0] for prop in ['contrast', 'energy', 'homogeneity'])

# Main processing
def process_all():
    imgs = glob.glob(os.path.join(INPUT_DIR, '**', '*.jpg'), recursive=True)
    for img_path in imgs:
        img = cv2.imread(img_path)
        if img is None:
            continue
        results = yolo_model(img)[0]
        person_boxes = [box.xyxy[0].cpu().numpy().astype(int) for box, cls in zip(results.boxes, results.boxes.cls) if int(cls) == 0]
        rel_dir = os.path.relpath(os.path.dirname(img_path), INPUT_DIR)
        out_dir = os.path.join(OUTPUT_DIR, rel_dir)
        os.makedirs(out_dir, exist_ok=True)
        for i, (x1, y1, x2, y2) in enumerate(person_boxes):
            crop = img[y1:y2, x1:x2]
            mask = get_person_mask_resnet(crop)
            landmarks = get_landmarks(crop)
            if not landmarks:
                continue
            upper, lower, hands, coords = split_parts(mask, landmarks)
            upper_feat = extract_clothing_features(crop, upper)
            lower_feat = extract_clothing_features(crop, lower)
            person_id_vector = np.concatenate([upper_feat, lower_feat])
            print(f"Person {i} features (Upper+Lower):", person_id_vector[:10])
            gray_crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
            contrast, energy, homogeneity = get_texture_features(gray_crop)
            print(f"Person {i+1} Texture - Contrast: {contrast}, Energy: {energy}, Homogeneity: {homogeneity})")
            vis = crop.copy()
            if 'LEFT_SHOULDER' in coords and 'RIGHT_SHOULDER' in coords:
                cv2.line(vis, coords['LEFT_SHOULDER'], coords['RIGHT_SHOULDER'], (0, 0, 255), 2)
            if 'LEFT_HIP' in coords and 'RIGHT_HIP' in coords:
                cv2.line(vis, coords['LEFT_HIP'], coords['RIGHT_HIP'], (0, 255, 0), 2)
            for side in ['LEFT', 'RIGHT']:
                key = f'{side}_WRIST'
                if key in coords:
                    cv2.circle(vis, coords[key], 5, (255, 0, 0), -1)
            basename = os.path.splitext(os.path.basename(img_path))[0]
            out_path = os.path.join(out_dir, f"{basename}_p{i}.jpg")
            cv2.imwrite(out_path, vis)
            hist_dir = os.path.join(out_dir, "histograms")
            os.makedirs(hist_dir, exist_ok=True)
            plot_histogram(crop, upper, f"Person {i+1} Upper HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_upper_hist.png"))
            plot_histogram(crop, lower, f"Person {i+1} Lower HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_lower_hist.png"))
            plot_histogram(crop, hands, f"Person {i+1} Hand HSV Histogram", os.path.join(hist_dir, f"{basename}_p{i}_hand_hist.png"))

if __name__ == '__main__':
    process_all()
    print("Processing complete. Segmented images saved to:", OUTPUT_DIR)


c:\Users\Administrator\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Administrator\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=LRASPP_MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\Administrator\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VO


0: 480x640 2 persons, 3 cars, 135.2ms
Speed: 10.9ms preprocess, 135.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 5 persons, 1 car, 132.6ms
Speed: 2.4ms preprocess, 132.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


ValueError: operands could not be broadcast together with shapes (41,19) (520,520) 